In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [ ]:
df = pd.DataFrame()
print(df)
pages = [str(i) for i in range(1,6)]
country_release=[]

Empty DataFrame
Columns: []
Index: []


In [ ]:
#Extracting information from multiple pages 

for page in pages:
  print("Importing page", page)
  url = "https://musicbrainz.org/search?query=2022&type=release&limit=100&method=direct&page=" + page 
  response=requests.get(url)
  sleep(randint(8,15))
  #clear_output(wait = True)

  soup = BeautifulSoup(response.text, 'html.parser')
  tbl=soup.find('table',{'class': 'tbl'})

  df_new = pd.read_html(str(tbl))[0]
  df = df.append(df_new)

  rows = tbl.find_all('tr')

  for row in range(1, len(rows)):
    td = rows[row].find_all('td')
    if len(td[4].find_all('li', {'class':'show-all'})) > 0:
      country_release.append("Multiple event releases")
    else:
      li = td[4].find_all('li', {'class':'release-event'})
      if len(li) > 0:
        country = li[0].find_all('abbr')
        if len(country) > 0:
          span = li[0].find_all('span', {'class':'release-date'})
          release_date = span[0].text
          attribute = country[0].attrs
          title = attribute.get('title')
          country_release.append(str(title)+str("/")+str(release_date))
        else:
          country_release.append("None")
      else:
        country_release.append("None")

df["Country/Date"] = country_release

Importing page 1
Importing page 2
Importing page 3
Importing page 4
Importing page 5


In [ ]:
df.shape

(500, 11)

In [ ]:
df

,Name,Artist,Format,Tracks,Country/Date,Label,Catalog#,Barcode,Language,Type,Status
0,2022,Bazanji,Digital Media,1,[Worldwide]/2022-01-07,Bazanji LLC,NaN,196512300490,eng / Latn,Single,Official
1,2022,Célio Azevedo,Digital Media,4,Brazil/2022-01-03,Célio Azevedo Produções (Prematório Records),NaN,5059806404098,ces / Qaaa,Soundtrack,Official
2,2022,big hamus,(unknown),10,[Worldwide]/2022-01-31,NaN,NaN,[none],eng / Latn,Album,Official
3,2022,Sofiabot,Digital Media,1,[Worldwide]/2021-12-31,End of 2021 Records,2021-7+8=2022,[none],zxx / Zmth,Single,Official
4,2022 (2022 - Single),King Jase,Digital Media,1,[Worldwide]/2022-01-15,EMPIRE (US label/distributor/publisher of hip ...,KING11422,195729303188,eng / -,Single,Official
...,...,...,...,...,...,...,...,...,...,...,...
95,Infected Cadaveric Slaves - Demo Tape 2022,Septic Vomet,Digital Media,6,[Worldwide]/2022-10-14,[no label] (Special purpose label – white labe...,NaN,NaN,eng / Latn,EP + Demo,Official
96,INTERACTIVE LIVE SHOW 2022 ZCON ストーリー,平沢進,Digital Media,1,Japan/2021-12-24,Chaos Union (Teslakite),NaN,[none],jpn / Jpan,Other + Spokenword,Official
97,Jaybird Music Project August 2022 Sampler,Jaybird Music Project,Digital Media,13,[Worldwide]/2022-08-23,Jaybird Music Project,NaN,[none],eng / Latn,Other + Compilation,Official
98,Jaybird Music Project March 2022 Sampler,Jaybird Music Project,Digital Media,13,[Worldwide]/2022-08-23,Jaybird Music Project,NaN,[none],eng / Latn,Other + Compilation,Official


In [ ]:
#renaming columns in the dataframe
df.rename(columns = {'Name':'ReleaseName', 'Country/Date':'Release_Country_Date'}, inplace = True)

In [ ]:
df.head(3)

,ReleaseName,Artist,Format,Tracks,Release_Country_Date,Label,Catalog#,Barcode,Language,Type,Status
0,2022,Bazanji,Digital Media,1,[Worldwide]/2022-01-07,Bazanji LLC,NaN,196512300490,eng / Latn,Single,Official
1,2022,Célio Azevedo,Digital Media,4,Brazil/2022-01-03,Célio Azevedo Produções (Prematório Records),NaN,5059806404098,ces / Qaaa,Soundtrack,Official
2,2022,big hamus,(unknown),10,[Worldwide]/2022-01-31,NaN,NaN,[none],eng / Latn,Album,Official


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ReleaseName           500 non-null    object
 1   Artist                500 non-null    object
 2   Format                500 non-null    object
 3   Tracks                500 non-null    object
 4   Release_Country_Date  500 non-null    object
 5   Label                 368 non-null    object
 6   Catalog#              151 non-null    object
 7   Barcode               311 non-null    object
 8   Language              500 non-null    object
 9   Type                  496 non-null    object
 10  Status                481 non-null    object
dtypes: object(11)
memory usage: 46.9+ KB


In [ ]:
df.describe()

,ReleaseName,Artist,Format,Tracks,Release_Country_Date,Label,Catalog#,Barcode,Language,Type,Status
count,500,500,500,500,500,368,151,311,500,496,481
unique,456,268,21,87,295,189,113,168,26,36,3
top,Demo 2022,Various Artists,Digital Media,1,None,[no label] (Special purpose label – white labe...,[none],[none],eng / Latn,Album + Compilation,Official
freq,16,147,374,96,46,78,25,130,339,146,434


In [ ]:
#counting duplicate rows in the dataframe
print("The dataset has", df.shape[0], "rows and", df.shape[1], "columns.")
dupRows = len(df)-len(df.drop_duplicates())
print("Out of the total,", dupRows, "duplicate records will be dropped.")

#Dropping Duplicate Rows 
df=df.drop_duplicates()
print("Dataset shape after removing duplicates is:")
df.shape

The dataset has 500 rows and 11 columns.
Out of the total, 2 duplicate records will be dropped.
Dataset shape after removing duplicates is:


(498, 11)

In [ ]:
# Checking missing values
print(df.isnull().sum())

ReleaseName               0
Artist                    0
Format                    0
Tracks                    0
Release_Country_Date      0
Label                   130
Catalog#                347
Barcode                 189
Language                  0
Type                      4
Status                   19
dtype: int64


In [ ]:
#Checking unique values in each column
unique_values = {}
for col in df.columns:
    unique_values[col] = df[col].value_counts().shape[0]

pd.DataFrame(unique_values, index=['unique value count']).transpose()

,unique value count
ReleaseName,456
Artist,268
Format,21
Tracks,87
Release_Country_Date,295
Label,189
Catalog#,113
Barcode,168
Language,26
Type,36


In [ ]:
#Saving data to a csv files
df.to_csv('music_releases.csv')